In [ ]:
from activite import build_data, al_fields, get, get_dfs, get_results

In [ ]:
from openfisca_france import CountryTaxBenefitSystem

base = CountryTaxBenefitSystem()
base.load_extension("openfisca_france_local")
res = get_results(base)
res

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import sys

sys.path.append("../technique")
from utils import *

In [ ]:
df = get_data(
    "/home/thomas/Nextcloud/CodeursEnLiberte/EMS/dee/DEE_20230719_Données activités QF.xlsx",
    get,
)

In [ ]:
df.Activite.unique()

In [ ]:
df.UNITE.unique()

In [ ]:
df["SERVICE"] = df.Activite + "_" + df.UNITE

In [ ]:
len(df.SERVICE.unique())

In [ ]:
df[["SERVICE", "NOMBRE"]].groupby(by="SERVICE").sum().sort_values(
    "NOMBRE", ascending=False
)

In [ ]:
df["SERVICE2"] = df.Activite.str.replace("AL .*", "AL", regex=True) + "_" + df.UNITE

In [ ]:
df[["SERVICE2", "NOMBRE"]].groupby(by="SERVICE2").sum().sort_values(
    "NOMBRE", ascending=False
)

In [ ]:
usage = (
    df[["SERVICE", "N° PER", "MOIS", "NOMBRE"]]
    .groupby(by=["N° PER", "MOIS", "SERVICE"])
    .sum()
)  # .reset_index()

In [ ]:
usage

In [ ]:
flat_usage = usage.reset_index()
flat_usage

In [ ]:
multiple = (
    flat_usage[["N° PER", "MOIS", "SERVICE"]].groupby(by=["N° PER", "MOIS"]).count()
)
multiple[multiple.SERVICE > 1].index

In [ ]:
data = flat_usage.join(
    multiple[multiple.SERVICE > 1], on=["N° PER", "MOIS"], rsuffix="r", how="inner"
).set_index(["N° PER", "MOIS"])

In [ ]:
with pd.option_context("display.max_rows", 300):
    display(data[0:100])

In [ ]:
data[data.SERVICE.str.contains("APM")][["SERVICE", "NOMBRE"]].groupby("SERVICE").count()